In [2]:
import data_extraction
import pandas as pd

pd.set_option('display.max_columns', None)

myObject = data_extraction.DataExtractor()
link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
df = myObject.retrieve_pdf_data(link)

type(df)

pandas.core.frame.DataFrame

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [3]:
df.head(5)

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [4]:
df.tail(5)

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
15315,14913,Stephen,Jenkins,1943-08-09,"Thornton, Carroll and Newman",s.jenkins@smith.com,Studio 41I\nJones lodge\nOliviaborough\nE8 3DU,United Kingdom,GB,+44(0)292018946,2016-04-15,2bd3a12f-a92d-4cdd-b99c-fc70572db302
15316,14994,Stephen,Smith,1948-08-20,Robinson-Harris,s.smith@smith.com,530 Young parkway\nMillsfurt\nL4G 7NX,United Kingdom,GB,+44(0)1144960977,2020-07-20,d234c04b-c07c-46a5-a902-526f91478ecc
15317,15012,Stephen,Losekann,1940-10-09,Rosenow,s.losekann@smith.com,Viviane-Fritsch-Straße 3/5\n15064 Bad Liebenwerda,Germany,DE,02984 08192,2021-03-07,1a0a8b7b-7c17-42d8-a946-8a85d5495651
15318,15269,Stephen,Rivera,1952-06-04,"Taylor, Fry and Jones",s.rivera@smith.com,"660 Ross Falls Suite 357\nAnthonymouth, MA 09610",United States,US,239.711.3836,2011-01-03,187fe06e-bd5f-4381-af2f-d7ac37ca7572
15319,1249,Stephen,Duncan,1994-03-27,"Phillips, Brown and Powell",s.duncan@smith.com,Studio 4\nHancock road\nPhillipsview\nST1X 3XB,United Kingdom,GB,+44(0)1314960870,2015-08-28,0589bbca-1d58-4b1f-9d0a-04ed4c57aaa1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [6]:
df.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

index seems to be an integer

In [7]:
df['index'] = df['index'].astype('int64')

We have some date columns

In [8]:
df['date_of_birth']=pd.to_datetime(df['date_of_birth'],infer_datetime_format=True, errors='coerce')
df['join_date']=pd.to_datetime(df['join_date'],infer_datetime_format=True, errors='coerce')

inspect some potentially categorical data:
Country
Country Code

In [9]:
df["country"].describe()

count              15320
unique                19
top       United Kingdom
freq                9371
Name: country, dtype: object

In [10]:
df["country_code"].describe()

count     15320
unique       20
top          GB
freq       9365
Name: country_code, dtype: object

Not bad, though there's 19 countries and 20 codes!

In [11]:
df["country"] = df["country"].astype("category")
df["country_code"] = df["country_code"].astype("category")

Companies as categories?

In [12]:
df["company"].describe()

count         15320
unique        12105
top       Smith Inc
freq             22
Name: company, dtype: object

Nope. What about duplicate lines?

In [13]:
df.duplicated().sum()

0

None! ok. Let's just check on the critical columns only - not system generated ones

In [14]:
df.duplicated(subset=["first_name", "last_name", "date_of_birth", "country"],keep="first").sum()

20

20 duplicates. There's nothing in the other columns to average or anything like that - hopefully dropping 20 won't kill us.

In [15]:
df.drop_duplicates(subset=["first_name", "last_name", "date_of_birth", "country"],keep="first",inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15300 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15300 non-null  int64         
 1   first_name     15300 non-null  object        
 2   last_name      15300 non-null  object        
 3   date_of_birth  15284 non-null  datetime64[ns]
 4   company        15300 non-null  object        
 5   email_address  15300 non-null  object        
 6   address        15300 non-null  object        
 7   country        15300 non-null  category      
 8   country_code   15300 non-null  category      
 9   phone_number   15300 non-null  object        
 10  join_date      15284 non-null  datetime64[ns]
 11  user_uuid      15300 non-null  object        
dtypes: category(2), datetime64[ns](2), int64(1), object(7)
memory usage: 1.3+ MB


standardise the phone numbers.

In [16]:
def standardise_phone_number(number: str):
    if number[:1] == "+" :
        number = number[1:]
    number = number.replace(" ","")
    number = number.replace("-","")

    if number[:2] != "00":
        number = "00" + number

for index, row in df.iterrows():
    row["phone_number"] = standardise_phone_number(row["phone_number"])




Should I do something about the null values? What about email parsing with regex?
what about that index column and the uuid one - are they still relevant?

Now put the code into a local database maybe

In [17]:
myObject.upload_to_db(df,"dim_users")